### Import neccesary Libraries

In [1]:
import numpy as np
import pickle
import tqdm

### Reading Dataset

In [2]:
train_dir = "./data/cifar-10-batches-py/"

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    X = dict[b'data']
    y = np.array(dict[b'labels'])
    return X, y

In [4]:
xs = []
ys = []
for b in range(1,6):
    f = train_dir+'data_batch_'+str(b)
    X, y = unpickle(f)
    xs.append(X)
    ys.append(y)
Xtr = np.concatenate(xs)
Ytr = np.concatenate(ys)
del X, y
del xs, ys

In [5]:
Xtr = Xtr/255

In [6]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Activation Functions

In [7]:
class ReLU:
    def __init__(self):
        self.cache = None
    
    def forward(self, X):
        self.cache = X
        return np.maximum(0, X)
    
    def backward(self, dY):
        X = self.cache
#         print(f'dim of X relu:{X.shape}, dim of dY relu: {dY.shape}')
        dY[X <= 0] = 0
        return dY

### Layers

### Class definition
Class `Conv2D` implements a 2D convolution layer

#### Constructor
The `Conv2D` class has constructor with following parameters:       
  - `in_ch` (int) : number of in channels
  - `out_ch` (int): number of out channels
  - `k_size` (int): dimension size of kernel
  - `stride` (int)
  - `padding` (string): valid or same


#### Methods

**`forward(arr_in)`**
Computes the forward pass of the layer given an input tensor `arr_in`. Returns the output tensor and caches values needed for the backward pass.

**`backward(out_grad)`**
Computes the backward pass of the layer given the gradient of the loss with respect to the output tensor `out_grad`. Returns the gradient of the loss with respect to the input tensor.

**`getGrads()`**
Returns the weights and bias gradients

In [25]:
class Conv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride = 1, padding = 'valid'):
        self.in_ch = in_channels
        self.out_ch = out_channels
        self.stride = stride
        self.padding = padding
        self.k_size = kernel_size
        self.w = np.random.randn(self.k_size, self.k_size, self.in_ch, self.out_ch)/10
#         print(self.w.shape)
        self.bias = np.random.randn(self.out_ch)/10
        
    '''
        arr_in  - input data of shape (n, h_in, w_in, in_ch)
        weights - convolutional filter weights of shape (k_size, k_size, in_ch, out_ch)
        out     - output feature map of shape (n, out_h, out_w, out_ch)
        b       - bias tensor with shape (out_ch, )
    '''
    def forward(self, arr_in):
        n, in_h, in_w, in_ch = arr_in.shape
        self.arr_in0 = arr_in
#         '''
#             output array dimension for convolution of two matrix is given by relation:
#                 floor((in_h+2*self.pad-self.k_size)/self.stride)+1, floor(in_w+2*self.pad-self.k_size)/self.stride)+1
#         '''
        if(self.padding == 'valid'):
            self.pad = (self.k_size - 1) // 2
            
        else:
            self.pad = 0
        out_h = (in_h+2*self.pad-self.k_size)//self.stride+1
        out_w = (in_w+2*self.pad-self.k_size)//self.stride+1

#         out_h = (in_h-self.k_size)//self.stride+1
#         out_w = (in_w-self.k_size)//self.stride+1
    
        
        arr_out = np.zeros((n, out_h, out_w, self.out_ch))
#         print(arr_out.shape)
        '''
            np.pad(array, pad_width)
                pad_width takes tuple of tuple, length of tuple equal to the axis in array indicating the number of pad to be added before and after of each axis
            we want to add padding in the image content only
        '''

        self.arr_pad = np.pad(arr_in, pad_width =  ((0,0), (self.pad, self.pad), (self.pad, self.pad), (0,0)))
#         print(self.arr_pad)
        # Convolution starts
        for i in range(0, out_h):
            for j in range(0, out_w):
#                 print(self.w.shape)
                temp = self.arr_pad[:,i*self.stride:self.k_size+i*self.stride,j*self.stride:self.k_size+j*self.stride,:,np.newaxis]*self.w[np.newaxis,:,:,:,:]
#                 print(np.sum(temp,axis=(1,2,3)).shape)
#                 print(arr_out[:,i,j,:].shape)
                arr_out[:,i,j,:] = np.sum(temp,axis=(1,2,3))
                
        return arr_out
    
    def backward(self, out_grad, lr=0.01):
        self.out_grad = out_grad
        n, hg_out, wg_out, _ = out_grad.shape
        n, h_in, w_in, _ = self.arr_in0.shape
        arr_in0_pad = self.arr_pad
#         print(arr_in0_pad.shape)
        self.in_grad = np.zeros(arr_in0_pad.shape)
        self.kernel_grad = np.zeros(self.w.shape)
        
        for i in range(0, hg_out):
            for j in range(0, wg_out):
                self.in_grad[:,i*self.stride:self.k_size+i*self.stride,j*self.stride:self.k_size+j*self.stride,:] += np.sum(self.w[np.newaxis, :, :, :, :]*out_grad[:, i:i+1, j:j+1, np.newaxis, :],axis=4)             
                self.kernel_grad += np.sum(arr_in0_pad[:, i*self.stride:i*self.stride+self.k_size, j*self.stride:j*self.stride+self.k_size, :, np.newaxis]*(out_grad[:, i:i+1, j:j+1, np.newaxis, :])/n,axis=0)
                
        self.w -= lr*self.kernel_grad/n
        self.bias -= lr*np.sum(out_grad/n, axis = (0,1,2))
        self.dbias = np.sum(out_grad, axis = (0,1,2))/n
#         self.kernel_grad = self.kernel_grad/n
        
        return self.in_grad[:, self.pad:self.pad+h_in, self.pad:self.pad+w_in, :]
    
    def update_weights(self, w, b):
        self.w = w
        self.bias = b
        
    def getGrads(self):
        return self.kernel_grad, self.dbias

### Class definition
Class `MaxPool2D` implements a 2D maxpooling layer

#### Constructor
The `MaxPool2D` class has constructor with following parameters:       
  - `k_size` (int) : kernel_size


#### Methods

**`forward(arr_in)`**
Performs a 2D max pooling operation on the input tensor with a given kernel size and stride

**`backward(out_grad)`**
The backward pass of MaxPool2d function takes as input the gradient tensor of the loss function with respect to the output of MaxPool2d, out_grad, and computes the gradient of the loss function with respect to the input of MaxPool2d.

In [9]:
class MaxPool2D:
    def __init__(self, kernel_size):
        self.k_size = kernel_size
        self.cache = None
    def forward(self, arr_in, stride=1):
#         print(arr_in.shape)
        self.stride = stride
        n, in_h, in_w, in_ch = arr_in.shape

        out_h = (in_h) // self.k_size
        out_w = (in_w) // self.k_size
        
        arr_out = np.max(arr_in.reshape(n, out_h, self.k_size, out_w, self.k_size, in_ch), axis=(2, 4))

        self.cache = arr_in, arr_out
        return arr_out
    
    def backward(self, out_grad):
        arr_in, arr_out = self.cache
        in_grad = (arr_in == np.repeat(np.repeat(arr_out, self.k_size, axis = 1), self.k_size, axis = 2))*(np.repeat(np.repeat(out_grad, self.k_size, axis = 1), self.k_size, axis = 2))
        return in_grad        

### Class definition
Class `Linear` implements fully connected neural network

#### Constructor
The `Linear` class has constructor with following parameters:       
  - `input_size` (int)
  - `output_size`(int)
  - `w` (np array) : weights
  - `b` (np array) : bias

#### Methods

**`forward(arr_in)`**
Performs a forward pass through the layer.

**`backward(out_grad)`**
Performs a backward pass through the layer.

In [24]:
class Linear:
    def __init__(self, input_size, output_size):
        self.w = np.random.randn(input_size, output_size) /10
        self.b = np.random.randn(1, output_size)/10
        self.cache = None
        
    def forward(self, arr_in):
        self.cache = arr_in
        return np.dot(arr_in, self.w) + self.b
    
    def backward(self, dY, lr=0.01):
        arr_in = self.cache        
        n = arr_in.shape[0]    
        d_arr_in = np.matmul(dY/n, self.w.T)
        self.dW = np.matmul(arr_in.T, dY/n)
        self.db = np.sum(dY/n, axis=0, keepdims=True)
        
        w_old = np.array(self.w, copy = True)
        self.w -= lr * self.dW
        self.b -= lr * self.db
        
        if((self.w == w_old).all()): print("equal")
        
        return d_arr_in
    
    def update_weights(self, w, b):
        self.w = w
        self.b = b
        
    def getGrads(self):
        return self.dW, self.b

### Class definition
Class `Flat` converts the shape of input between fully connected and convolution network

#### Methods

**`forward(arr_in)`**
Flatten 4D tensor to 1D array

**`backward(out_grad)`**
convert 1D array back to 4D tensor

In [11]:
class Flat():

    def __init__(self):
        self.shape = None
        
    def forward(self, arr_in):
#         print(arr_in.shape)
        self.shape = arr_in.shape
        return np.reshape(arr_in.ravel(), (arr_in.shape[0], -1))
    
    def backward(self, out_grad):
        return np.reshape(out_grad, self.shape)

### Optimizers

In [12]:
class Adam_opt:
    def __init__(self, lr, beta1 = 0.9, beta2 = 0.99, epsilon = 1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m_c = None
        self.v_c = None
        self.m_f = None
        self.v_f = None
        self.t = 0
        
    def update(self, conv_layers, fc_layers):
        self.t+=1
        if self.m_c is None:
            self.m_c = {}
            self.v_c = {}
            for i, layer in enumerate(conv_layers):
                kernel_grad, dbias = layer.getGrads()
                self.m_c[i] = (np.zeros_like(kernel_grad), np.zeros_like(dbias))
                self.v_c[i] = (np.zeros_like(kernel_grad), np.zeros_like(dbias))
        for i, layer in enumerate(conv_layers):
            kernel_grad, dbias = layer.getGrads()
            self.m_w, self.m_bias = self.m_c[i]
            self.v_w, self.v_bias = self.v_c[i]
            
            self.m_w = self.beta1 * self.m_w + (1 - self.beta1) * kernel_grad
            self.v_w = self.beta2 * self.v_w + (1 - self.beta2) * (kernel_grad ** 2)
            m_hat = self.m_w / (1 - self.beta1 ** self.t)
            v_hat = self.v_w / (1 - self.beta2 ** self.t)
            layer.w -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
            
            
            self.m_bias = self.beta1 * self.m_bias + (1 - self.beta1) * dbias
            self.v_bias = self.beta2 * self.v_bias + (1 - self.beta2) * (dbias ** 2)
            m_hat = self.m_bias / (1 - self.beta1 ** self.t)
            v_hat = self.v_bias / (1 - self.beta2 ** self.t)
            layer.bias -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
            self.m_c[i] = (self.m_w, self.m_bias)
            self.v_c[i] = (self.v_w, self.v_bias)
            
            
        if self.m_f is None:
            self.m_f = {}
            self.v_f = {}
            for i, layer in enumerate(fc_layers):
                dw, dbias = layer.getGrads()
                self.m_f[i] = (np.zeros_like(dw), np.zeros_like(dbias))
                self.v_f[i] = (np.zeros_like(dw), np.zeros_like(dbias))
                
        for i, layer in enumerate(fc_layers):
            dw, dbias = layer.getGrads()
            self.m_w, self.m_bias = self.m_f[i]
            self.v_w, self.v_bias = self.v_f[i]
            
            self.m_w = self.beta1 * self.m_w + (1 - self.beta1) * kernel_grad
            self.v_w = self.beta2 * self.v_w + (1 - self.beta2) * (kernel_grad ** 2)
            m_hat = self.m_w / (1 - self.beta1 ** self.t)
            v_hat = self.v_w / (1 - self.beta2 ** self.t)
            layer.w -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
            
            
            self.m_bias = self.beta1 * self.m_bias + (1 - self.beta1) * dbias
            self.v_bias = self.beta2 * self.v_bias + (1 - self.beta2) * (dbias ** 2)
            m_hat = self.m_bias / (1 - self.beta1 ** self.t)
            v_hat = self.v_bias / (1 - self.beta2 ** self.t)
            layer.b -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
            
            self.m_f[i] = (self.m_w, self.m_bias)
            self.v_f[i] = (self.v_w, self.v_bias)

### Loss Function

In [13]:
class CategoricalCrossentropy:
    def __init__(self):
        pass
    
    def __call__(self, y_pred, y_true):
        
        loss = -np.sum(y_true * np.log(np.clip(y_pred, 1e-20, 1 - 1e-20))) / y_pred.shape[0]        
        return loss
    
    def backward(self, y_pred, y_true):

        d_y_pred = -(y_true / np.clip(y_pred, 1e-20, 1 - 1e-20)) / y_pred.shape[0]        
        return d_y_pred

### CNN

In [26]:
class CNN:
    def __init__(self):
        self.conv1 = Conv2D(3, 32, 3)
        self.pool1 = MaxPool2D(2)
        self.conv2 = Conv2D(32, 64, 5)
        self.pool2 = MaxPool2D(2)
        self.conv3 = Conv2D(64, 64, 3)
        self.fc1 = Linear(64*8*8, 64)
#         self.fc1 = Linear(65536, 64)
        self.fc2 = Linear(64, 10)
        self.relu1 = ReLU()
        self.relu2 = ReLU()
        self.relu3 = ReLU()
        self.relu4 = ReLU()
        self.flatten = Flat()
        
        self.conv_layers = [self.conv1, self.conv2, self.conv3]
        self.fc_layers = [self.fc1, self.fc2]
        
    def forward(self, X):
        y = self.conv1.forward(X)
        y = self.pool1.forward(self.relu1.forward(y))        
        y = self.pool2.forward(self.relu2.forward(self.conv2.forward(y)))
        y = self.relu3.forward(self.conv3.forward(y))
        y = self.flatten.forward(y)     # Flatten
        y = self.relu4.forward(self.fc1.forward(y))
        y = self.fc2.forward(y)
        return y
        
        
    def backward(self, loss):
        grad = self.fc2.backward(loss)
        grad = self.relu4.backward(grad)
        grad = self.fc1.backward(grad)
        grad = self.flatten.backward(grad)
        grad = self.relu3.backward(grad)
        grad = self.conv3.backward(grad)
        grad = self.pool2.backward(grad)
        grad = self.relu2.backward(grad)
        grad = self.conv2.backward(grad)
        grad = self.pool1.backward(grad)
        grad = self.relu1.backward(grad)
        grad = self.conv1.backward(grad)
        return grad
    
    def train(self, X_train, y_train, epochs, batch_size, lr=0.001):
        optimizer = Adam_opt(lr)        
        loss_func = CategoricalCrossentropy()
        
        n_batches = X_train.shape[0] // batch_size
        print(n_batches)

        arr_train_loss = []
        arr_train_acc = []
        val_loss = []
        val_acc = []
        
        for epoc in range(epochs):
            print(epoc)
            train_loss = 0
            train_acc = 0
            
            for i in tqdm.tqdm(range(n_batches)):

                x_batch = X_train[i*batch_size:(i+1)*batch_size]
                y_batch = y_train[i*batch_size:(i+1)*batch_size]

                # Forward pass
#                 print("Forward")
                out = self.forward(x_batch)
                loss = loss_func(out, y_batch)

                # Backward pass
#                 grad = softmax_cross_entropy(out, y_batch)
#                 print("Backward")
#                 grad = loss_func.backward(out, y_batch)
                grad = out - y_batch
                self.backward(grad)

                # Update weights
#                 optimizer.update(self.conv_layers, self.fc_layers)

                # Calculate accuracy
                pred = np.argmax(out, axis=1)
                acc = np.mean(pred == np.argmax(y_batch, axis=1))

                train_loss += loss
                train_acc += acc

            train_loss /= n_batches
            train_acc /= n_batches

            arr_train_loss.append(train_loss)
            arr_train_acc.append(train_acc)

        return arr_train_loss, arr_train_acc
    
    def predict(self, X_test, y_test):
        
        y_pred = self.forward(X_test)
        y_pred_labels = np.argmax(y_pred, axis=1)
    
        num_classes = len(np.unique(y_true))
        total = np.zeros(num_classes)
        correct = np.zeros(num_classes)

        for i in range(len(y_test)):
            true_label = y_test[i]
            pred_label = y_pred_labels[i]
            total[true_label] += 1
            if (true_label == pred_label):
                correct[true_label] += 1

        class_acc = {}
        for i in range(num_classes):
            if total[i] == 0:
                class_acc[i] = 0.0
            else:
                class_acc[i] = correct[i] / total[i]

        return class_acc

In [27]:
model = CNN()

In [28]:
Xtr = np.reshape(Xtr, (-1, 3, 32, 32))[0:100]
Xtr = np.transpose(Xtr, (0, 2, 3, 1))[0:100]

In [29]:
Xtr.shape

(100, 32, 32, 3)

In [30]:
# model.forward(Xtr)

In [31]:
def convert_categorical2one_hot(y: np.array) -> np.array:
    """
    :param y - categorical array with (n, 1) shape
    :return one hot array with (n, k) shape
    ----------------------------------------------------------------------------
    n - number of examples
    k - number of classes
    """
    one_hot_matrix = np.zeros((y.size, y.max() + 1))
    one_hot_matrix[np.arange(y.size), y] = 1
    return one_hot_matrix

In [32]:
y_train = convert_categorical2one_hot(Ytr)

In [34]:
model.train(Xtr, y_train, 100, 50, 0.1)

2
0


100%|████████████████████████████████████████████████████████████████| 2/2 [00:16<00:00,  8.37s/it]


1


100%|████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.88s/it]


2


...

98


100%|████████████████████████████████████████████████████████████████| 2/2 [00:18<00:00,  9.50s/it]


99


100%|████████████████████████████████████████████████████████████████| 2/2 [00:18<00:00,  9.44s/it]


([18.766169072431282,
  18.728615198320497,
  18.711270391114105,
  17.869658173393606,
  17.437949437940002,
  16.608729302886175,
  16.5561472275392,
  16.100467549206492,
  15.673703895872347,
  14.857180472550588,
  14.440469137551721,
  13.938063930255161,
  13.089683409355555,
  13.034285736637287,
  12.996002687182246,
  12.965142169174149,
  12.548349422401643,
  12.11609784392467,
  11.67786434706439,
  11.643396633138089,
  11.617395637035449,
  11.195445032366623,
  10.778855832538099,
  10.348731917689681,
  9.932307324287049,
  9.88979148410112,
  9.86359466982842,
  9.842259357182707,
  9.823796975468712,
  9.407408545156649,
  9.384369286612419,
  8.97359192500333,
  8.947759147128725,
  8.928501418849827,
  8.911954759081091,
  8.897333258169898,
  8.494328151780703,
  8.473282975446578,
  8.457001580283656,
  8.442898223534467,
  8.43015920807994,
  8.418456789578741,
  8.407564738335884,
  8.397305071662535,
  8.387594653634977,
  8.378369748026394,
  7.58498470427892

In [ ]:
# def softmax_accuracy(y_hat: np.array, y: np.array) -> float:
#     """
#     :param y_hat - 2D one-hot prediction tensor with shape (n, k)
#     :param y - 2D one-hot ground truth labels tensor with shape (n, k)
#     ----------------------------------------------------------------------------
#     n - number of examples in batch
#     k - number of classes
#     """
#     y_hat = convert_prob2one_hot(y_hat)
#     return (y_hat == y).all(axis=1).mean()


def softmax_cross_entropy(y_hat, y, eps=1e-20) -> float:
    """
    :param y_hat - 2D one-hot prediction tensor with shape (n, k)
    :param y - 2D one-hot ground truth labels tensor with shape (n, k)
    ----------------------------------------------------------------------------
    n - number of examples in batch
    k - number of classes
    """
    n = y_hat.shape[0]
    return - np.sum(y * np.log(np.clip(y_hat, eps, 1.))) / n

In [11]:
a = np.random.randn(1,5,5,3)
k = np.random.randn(2,2,3,1)

conv_layer = Conv2D(in_channels=3, out_channels=1, kernel_size=2, stride=1, padding=0)
# conv_layer.w = k
output_matrix = conv_layer.forward(a)
print("Forward pass output:\n", output_matrix.shape)
d_output_matrix = np.ones((1,4,4,1))
d_input_matrix = conv_layer.backward(d_output_matrix, lr=0.001)
print("Backward pass output:\n", d_input_matrix.shape)

Forward pass output:
 (1, 4, 4, 1)
Backward pass output:
 (1, 5, 5, 3)


In [12]:
a = np.random.randn(1,8,8,2)
k = np.random.randn(2,2,3,1)

pool = MaxPool2D(2)
# conv_layer.w = k
output_matrix = pool.forward(a)
print("Forward pass output:\n", output_matrix.shape)
d_output_matrix = np.ones((1,4,4,2))
d_input_matrix = pool.backward(d_output_matrix)
print("Backward pass output:\n", d_input_matrix.shape)

Forward pass output:
 (1, 4, 4, 2)
Backward pass output:
 (1, 8, 8, 2)


In [ ]:
'''
    To implement CNN following functions will be needed
        1. Conv2D
            a. forward
            b. backward
            c. update weights
            d. get gradients
        2. MaxPool
            a. forward
            b. backward
        3. ReLU
            a. forward
            b. backward
        4. Softmax
            a. forward
            b. backward
        5. Full connected NN
            a. forward
            b. backward
            c. update wights
            d. get gradients
        6. Adam optimizers(to update weights)
'''


'''
    class ConvNet will include
        1. forward
        2. backward
        3. train
        4. predict

'''